## Custom blockchain built with Python

**Ángel C.**


### Libraries required to build this blockchain 

* **datetime**

* **hashlib**

* **json**

* **flask**

* **flask-ngrok**

## Installation

In [1]:
!pip install flask==0.12.2

     |████████████████████████████████| 83 kB 1.3 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [2]:
!pip install flask-ngrok==0.0.25

## Blockchain set up

In [3]:
# Importación de las librerías
import datetime
import hashlib
import json
from flask       import Flask, jsonify
from flask_ngrok import run_with_ngrok

**Essential blockchain methods:**     
* New block creation
* Get new block hash
* Consensus protocol: Proof of Work (PoW)
* Block hash generation
* Blockchain validity verification

In [5]:
# Blockchain creation
class Blockchain:
    
  def __init__(self):
    """ Constructor for class Blockchain. """

    self.chain = []
    self.create_block(proof = 1, previous_hash = '0')
      
  
  def create_block(self, proof, previous_hash):
    """ New block creation. 

      Arguments:
        - proof: Nonce of current block. (proof != hash)
        - previous_hash: Hash of previous block.

      Returns: 
        - block: New block creation. 
      """

    block = { 'index'         : len(self.chain)+1,
              'timestamp'     : str(datetime.datetime.now()),
              'proof'         : proof,
              'previous_hash' : previous_hash}
    self.chain.append(block)
    return block

  def get_previous_block(self):
    """ Blockchain's previous block .
    
      Returns:
        - Last block. """

    return self.chain[-1]
  
  def proof_of_work(self, previous_proof):
    """ Consensus protocol Proof of Work (PoW).
    
      Arguments:
        - previous_proof: Nonce of previous block.

      Returns:
        - new_proof: Returns new nonce obtained with PoW. """

    new_proof = 1
    check_proof = False
    while check_proof is False:
        hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] == '0000':
            check_proof = True
        else: 
            new_proof += 1
    return new_proof
  
  def hash(self, block):
    """ Calculation of a block's hash.
    
    Arguments:
        - block: Identifies a block in the Blockchain.
    
    Returns:
        - hash_block: Returns block's hash """

    encoded_block = json.dumps(block, sort_keys = True).encode()
    hash_block = hashlib.sha256(encoded_block).hexdigest()
    return hash_block
  
  def is_chain_valid(self, chain):
    """ Determines if the Blockchain is valid. 
    
    Arguments:
        - chain: Chain of blocks containing transactional information.
    
    Returns:
        - True/False: Blockchain validity """

    previous_block = chain[0]
    block_index = 1
    while block_index < len(chain):
        block = chain[block_index]
        if block['previous_hash'] != self.hash(previous_block):
            return False
        previous_proof = previous_block['proof']
        proof = block['proof']
        hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] != '0000':
            return False
        previous_block = block
        block_index += 1
    return True

Web app (REST API):

REST API requests: 
* **mine_block()**
* **get_chain()**
* **is_valid()**

In [6]:
# Web app creation
# Running the app on Notebook
app = Flask(__name__)
run_with_ngrok(app)  

# If the response is 500, update flask and run the next line
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

# Blockchain creation
blockchain = Blockchain()


@app.route('/mine_block', methods=['GET'])
def mine_block():
  """ New block mining """

  previous_block  = blockchain.get_previous_block()
  previous_proof  = previous_block['proof']
  proof           = blockchain.proof_of_work(previous_proof)
  previous_hash   = blockchain.hash(previous_block)
  block           = blockchain.create_block(proof, previous_hash)
  response = {'message'       : 'A new block has been mined!', 
              'index'         : block['index'],
              'timestamp'     : block['timestamp'],
              'proof'         : block['proof'],
              'previous_hash' : block['previous_hash']}
  return jsonify(response), 200

@app.route('/get_chain', methods=['GET'])
def get_chain():
  """ Get blockchain data """
  response = {'chain'   : blockchain.chain, 
              'length'  : len(blockchain.chain)}
  return jsonify(response), 200

@app.route('/is_valid', methods = ['GET'])
def is_valid():
  """ Check if the blockchain is valid """

  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
      response = {'message' : 'Blockchain is valid!'}
  else:
      response = {'message' : 'Blockchain NOT valid!'}
  return jsonify(response), 200  

In [ ]:
# To run the app on Google colab
app.run()

# To run the app outside Google colab
# app.run(host = '0.0.0.0', port = 5000)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3e11-34-125-199-38.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/Feb/2022 21:04:53] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:05:24] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:05:43] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:05:50] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:05:54] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:06:11] "GET /is_valid HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:07:06] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:07:07] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:07:08] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:07:09] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:07:10] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:07:23] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:08:44] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022 21:08:45] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2022